In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [3]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        print('config_filepath:', config_filepath)
        self.config = read_yaml(config_filepath)
        print('self.config:', self.config)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        print('config:', config)
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])   
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column= schema.name
        )
        
        return model_evaluation_config

In [4]:
import os
import pandas as pd 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib


In [5]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae  = mean_absolute_error(actual, pred)
        r2   = r2_score(actual, pred)
        return rmse, mae, r2
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model     = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        predicted_qualities = model.predict(test_x)
        
        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
        
        #save metrics as json
        scores = {'rmse': rmse, 'mae':mae, 'r2':r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)    

In [6]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    Model_evaluation = ModelEvaluation(config=model_evaluation_config)
    Model_evaluation.save_results()
    
except Exception as e:
    print(e)

config_filepath: config\config.yaml
[2025-03-06 06:38:39,237: INFO: Yaml file config\config.yaml loaded successfully]
self.config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://www.kaggle.com/api/v1/datasets/download/uciml/red-wine-quality-cortez-et-al-2009', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_